In [2]:
import cv2
import tensorflow as tf
import os
import sys
sys.path.append('./Swin-Transformer-TF')
from swintransformer import SwinTransformer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from vit_keras import vit, utils, visualize

print("num gpu:", len(tf.config.experimental.list_physical_devices('GPU')))

/home/afrida/anaconda3/envs/skin_cancer_classification/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.8.0 and strictly below 2.11.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [6]:
x_train = np.load("./breaSwinTNet/x_train.npy")
y_train=np.load("./breaSwinTNet/y_train.npy")

In [8]:
IMAGE_SIZE = [x_train.shape[1], x_train.shape[1], 3]
NUM_CLASSES = 7

swin_vit = SwinTransformer('swin_tiny_224', include_top=False, pretrained=True)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape = IMAGE_SIZE))
model.add(swin_vit)
model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

model.build(IMAGE_SIZE)

2022-11-30 19:11:36.195376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.summary()

model.compile(optimizer = tf.keras.optimizers.Adadelta(learning_rate = 0.1), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 swin_tiny_224 (SwinTransfor  (None, 768)              27769058  
 merModel)                                                       
                                                                 
 dense (Dense)               (None, 7)                 5383      
                                                                 
Total params: 27,774,441
Trainable params: 27,524,737
Non-trainable params: 249,704
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, batch_size = 4, epochs = 15, validation_split = 0.10)

Epoch 1/15


2022-11-30 19:11:54.685279: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 3256221696 exceeds 10% of free system memory.


1352/1352 [==============================] - ETA: 0s - loss: 0.7802 - accuracy: 0.7311WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fdb1c46e430> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-11-30 19:29:25.815341: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 361869312 exceeds 10% of free system memory.


1352/1352 [==============================] - 1102s 786ms/step - loss: 0.7802 - accuracy: 0.7311 - val_loss: 0.5551 - val_accuracy: 0.7820
Epoch 2/15
1352/1352 [==============================] - 1017s 753ms/step - loss: 0.5303 - accuracy: 0.8077 - val_loss: 0.3954 - val_accuracy: 0.8453
Epoch 3/15
1352/1352 [==============================] - 1007s 745ms/step - loss: 0.3668 - accuracy: 0.8650 - val_loss: 0.4696 - val_accuracy: 0.8386
Epoch 4/15
 781/1352 [================>.............] - ETA: 7:17 - loss: 0.2253 - accuracy: 0.9117

In [ ]:
x_test = np.load("./x_test.npy")
y_test=np.load("./y_test.npy")

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_prob = np.argmax(y_pred, axis = 1)

In [ ]:
import sklearn

print(f'Accuracy {sklearn.metrics.accuracy_score(y_test, y_prob)}')
AUC = sklearn.metrics.roc_auc_score(y_test, y_pred, multi_class='ovo')
print(f'AUC is {AUC}')
target_names = ["akiec", "bcc", "bkl", "df", "mel", "vasc", "nv"]
print('F1-score', sklearn.metrics.f1_score(y_test, y_prob, average = 'micro'))
print(f'Balanced accuracy {sklearn.metrics.balanced_accuracy_score(y_test, y_prob)}')
print(f'MCC {sklearn.metrics.matthews_corrcoef(y_test, y_prob)}')
print(f'{sklearn.metrics.classification_report(y_test, y_prob, target_names=target_names)}')